<a href="https://colab.research.google.com/github/aKamrani/persian-lawyer/blob/main/persian_lawyer_BETA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -qq install langchain langchain-core langchain-community hazm scikit-learn
!pip install gradio ollama langchain chromadb colab-xterm
!pip uninstall -y huggingface_hub
!pip install -U huggingface_hub
!pip install -Uqqq
!pip install bitsandbytes accelerate
!pip install accelerate transformers import os os._exit(00)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 987.6/987.6 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 371.7/371.7 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 892.6/892.6 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 41.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 71.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 80.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.8/238.8 kB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 18.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into ac

In [2]:
# import ollama
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain_community.document_loaders import WebBaseLoader
# from langchain_community.vectorstores import Chroma
# from langchain_community.embeddings import OllamaEmbeddings
# from langchain_community.llms import Ollama

# Create Ollama embeddings and vector store
# embeddings = OllamaEmbeddings(model="nomic-embed-text")
# vectorstore = Chroma.from_texts(texts=splits, embedding=embeddings)
# retriever = vectorstore.as_retriever()


from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from transformers import AutoModelForCausalLM , AutoTokenizer, GenerationConfig
import os, requests, torch
import gradio as gr
from hazm import *

model_name_or_id = "MehdiHosseiniMoghadam/AVA-Llama-3-V2"
llama3_model = AutoModelForCausalLM.from_pretrained(model_name_or_id, torch_dtype=torch.float16, device_map="auto", low_cpu_mem_usage=True)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_id)

# read crawled docs from my github repo
documents = []
for i in range(1,58):
  url = f'https://raw.githubusercontent.com/aKamrani/persian-lawyer/main/rules/rule-{i}.txt'
  response = requests.get(url)
  if response.status_code == 200:
    documents.append(response.text)
  else:
    print('failes')

print(len(documents))
# remove stop words
filtered_documents = [' '.join([word for word in word_tokenize(doc.lower()) if word not in stopwords_list()]) for doc in documents]

# word stemming
stemmer = Stemmer()
stemmed_documents = [' '.join([stemmer.stem(word) for word in word_tokenize(doc)]) for doc in filtered_documents]

# calculate TF-IDF scores
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(stemmed_documents)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/730 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/51.2k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/419 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


57


In [16]:
model_generation_config = GenerationConfig(
    do_sample=True,
    top_k=1,
    temperature=0.9,
    max_new_tokens=500,
    pad_token_id=tokenizer.eos_token_id
)

def tfidf_docs_retriever(query, num_documents = 5):
    num_documents = -1 * num_documents
    filtered_query_words = [word for word in word_tokenize(query.lower()) if word not in stopwords_list()]
    stemmed_query = ' '.join([stemmer.stem(word) for word in filtered_query_words])
    query_vector = vectorizer.transform([stemmed_query])
    similarity_scores = (tfidf_matrix * query_vector.T).toarray()
    top_similar_documents_indices = similarity_scores.flatten().argsort()[num_documents:][::-1]
    top_similar_documents = [documents[i] for i in top_similar_documents_indices]
    print(top_similar_documents_indices)
    return top_similar_documents

# def ollama_llm(question, context):
#     instruct = 'به فارسی جواب بده و فقط بر اساس متن زیر به سوال جواب بده:'
#     formatted_prompt = f"{question}\n\n{instruct}\n\n{context}"
#     response = ollama.chat(
#         model="llama3", messages=[{"role": "user", "content": formatted_prompt}]
#     )
#     return response["message"]["content"]

def llama3_fa_llm(question, context):
    # instruct = 'با توجه به متن زیر به این سوال پاسخ بده:'
    # formatted_prompt = f"{instruct}\n{question}\n\n{context}"
    formatted_prompt = f"{context}\n\n{question}"
    inputs = tokenizer(formatted_prompt, return_tensors="pt").to("cuda")
    outputs = llama3_model.generate(**inputs, generation_config=model_generation_config)
    ans = tokenizer.decode(outputs[0], skip_special_tokens=True).replace(formatted_prompt, '')
    return ans

def rag_chain(question):
    # retrieved_docs = retriever.invoke(question)
    retrieved_docs = tfidf_docs_retriever(question, 3)
    formatted_context = "\n\n".join(doc for doc in retrieved_docs)
    return llama3_fa_llm(question, formatted_context)

# gradio interface
def get_important_facts(question):
    return rag_chain(question)

# gradio app
iface = gr.Interface(
    fn=get_important_facts,
    inputs=gr.Textbox(lines=2, placeholder="سوال حقوقی خود را اینجا بنویسید..."),
    outputs="text",
    title="مشاوره حقوقی مالیات بر ارزش افزوده - RAG & Llama3",
    description="مشاوره حقوقی ماالیات بر ارزش افزوده",
)

In [15]:
question = 'چه کالاها و خدماتی از پرداخت مالیات معاف هستند ؟'
retrieved_docs = tfidf_docs_retriever(question, 2)
context = "\n\n".join(doc for doc in retrieved_docs)
formatted_prompt = f"{context}\n\n{question}"
inputs = tokenizer(formatted_prompt, return_tensors="pt").to("cuda")
outputs = llama3_model.generate(**inputs, generation_config=model_generation_config)
print(tokenizer.decode(outputs[0], skip_special_tokens=True).replace(formatted_prompt, ''))

[56 55]

کالاهای زیر از پرداخت مالیات بر ارزش افزوده معاف هستند:

1. محصولات کشاورزی فرآوری نشده، مانند محصولات خام زراعی و باغی، گیاهان دارویی، محصولات مرتعی، محصولات جنگل (از جمله چوب خام)، محصولات گلخانه (از جمله سبزی، صیفی، گل و گیاه و انواع قارچ)
2. دام زنده و خوراک آن، مانند دام زنده و خوراک آن که برای امور تغذیه انسان یا دام و فعالیت های اقتصادی، تولیدی و آزمایشگاهی، تولید، نگهداری و پرورش داده می شود.
3. بذر، نشاء، نهال، سم و کود
4. آب مصارف کشاورزی
5. کالاهای زیر:
   - شیر، پنیر و ماست
   - تخم ماکیان
   - آرد و نان
   - انواع گوشت و فرآورده های گوشتی
   - برنج، حبوبات، سویا و پروتئین سویا
   - انواع روغن های خوراکی، اعم از گیاهی و حیوانی
   - شیرخشک مخصوص تغذیه کودکان
   - تخم مرغ نطفه دار و تبدیل آن به جوجه یک روزه
6. کالاهای زیر:
   - خمیر کاغذ و کاغذ باطله
   - دفتر تحریر
   - کاغذ چاپ، تحریر و روزنامه
7. نسخه های کاغذی و الکترونیکی محصولات زیر:
   - کتاب
   - روزنامه
   - مجله و نشریه
8. کالاهای وارده همراه مسافر برای استفاده شخصی تا میزان معافیت مقرر طبق قانون امور گمرکی

In [17]:
question = 'آیا کشمش از پرداخت مالیات معاف هست ؟'
retrieved_docs = tfidf_docs_retriever(question, 2)
context = "\n\n".join(doc for doc in retrieved_docs)
formatted_prompt = f"{context}\n\n{question}"
inputs = tokenizer(formatted_prompt, return_tensors="pt").to("cuda")
outputs = llama3_model.generate(**inputs, generation_config=model_generation_config)
print(tokenizer.decode(outputs[0], skip_special_tokens=True).replace(formatted_prompt, ''))

[55 56]

بله، کشمش از پرداخت مالیات بر ارزش افزوده معاف است. این ماده در ماده ۹ قانون مالیات بر ارزش افزوده جمهوری اسلامی ایران ذکر شده است.assistant

بله، کشمش از پرداخت مالیات بر ارزش افزوده معاف است. این ماده در ماده ۹ قانون مالیات بر ارزش افزوده جمهوری اسلامی ایران ذکر شده است.assistant

بله، کشمش از پرداخت مالیات بر ارزش افزوده معاف است. این ماده در ماده ۹ قانون مالیات بر ارزش افزوده جمهوری اسلامی ایران ذکر شده است.assistant

بله، کشمش از پرداخت مالیات بر ارزش افزوده معاف است. این ماده در ماده ۹ قانون مالیات بر ارزش افزوده جمهوری اسلامی ایران ذکر شده است.assistant

بله، کشمش از پرداخت مالیات بر ارزش افزوده معاف است. این ماده در ماده ۹ قانون مالیات بر ارزش افزوده جمهوری اسلامی ایران ذکر شده است.assistant

بله، کشمش از پرداخت مالیات بر ارزش افزوده معاف است. این ماده در ماده ۹ قانون مالیات بر ارزش افزوده جمهوری اسلامی ایران ذکر شده است.assistant

بله، کشمش از پرداخت مالیات بر ارزش افزوده معاف است. این ماده در ماده ۹ قانون مالیات بر ارزش افزوده جمهوری اسلامی ایران ذکر شده است.assistan

In [18]:
question = 'مالیات و عوارض فروش را تعریف کن'
retrieved_docs = tfidf_docs_retriever(question, 2)
context = "\n\n".join(doc for doc in retrieved_docs)
formatted_prompt = f"{context}\n\n{question}"
inputs = tokenizer(formatted_prompt, return_tensors="pt").to("cuda")
outputs = llama3_model.generate(**inputs, generation_config=model_generation_config)
print(tokenizer.decode(outputs[0], skip_special_tokens=True).replace(formatted_prompt, ''))

[ 0 33]
ید؟

مالیات و عوارض فروش، مالیات و عوارض است که به موجب قانون مالیات بر ارزش افزوده به مودی تعلق می  گیرد و بر کالاها و خدمات مشمول مالیات و عوارض توسط او در یک دوره معین تعلق می  گیرد. این مالیات و عوارض به مودی در همان دوره پرداخت می  شود، مگر اینکه مودی از پرداخت آن به دلیل نسیه بودن معامله یا قرارداد استفاده کند...........assistant

مالیات و عوارض فروش، مالیات و عوارض است که به موجب قانون مالیات بر ارزش افزوده به مودی تعلق می  گیرد و بر کالاها و خدمات مشمول مالیات و عوارض توسط او در یک دوره معین تعلق می  گیرد. این مالیات و عوارض به مودی در همان دوره پرداخت می  شود، مگر اینکه مودی از پرداخت آن به دلیل نسیه بودن معامله یا قرارداد استفاده کند.assistant

مالیات و عوارض فروش، مالیات و عوارض است که به موجب قانون مالیات بر ارزش افزوده به مودی تعلق می  گیرد و بر کالاها و خدمات مشمول مالیات و عوارض توسط او در یک دوره معین تعلق می  گیرد. این مالیات و عوارض به مودی در همان دوره پرداخت می  شود، مگر اینکه مودی از پرداخت آن به دلیل نسیه بودن معامله یا قرارداد استفاده کند..assistant

مالی

In [19]:
question = ' تاریخ تعلق مالیات در معاملات غیرنقدی چه زمانی هست؟'
retrieved_docs = tfidf_docs_retriever(question, 2)
context = "\n\n".join(doc for doc in retrieved_docs)
formatted_prompt = f"{context}\n\n{question}"
inputs = tokenizer(formatted_prompt, return_tensors="pt").to("cuda")
outputs = llama3_model.generate(**inputs, generation_config=model_generation_config)
print(tokenizer.decode(outputs[0], skip_special_tokens=True).replace(formatted_prompt, ''))

[22 33 41  6  4]


ماده ۴ قانون مالیات بر ارزش افزوده می گوید: در معاملات غیرنقدی نظیر فروش اقساطی و اجاره به شرط تملیک و قراردادهای پیمانکاری و مشاوره ای، تاریخ تعلق مالیات و عوارض همان تاریخ صدور صورتحساب است؛ لکن مودی با رعایت تبصره فوق مجاز است پرداخت مالیات و عوارض فروش این نوع معاملات را تا زمان پرداخت ثمن معامله توسط خریدار یا مبلغ قرارداد توسط کارفرما، متناسباً، به تاخیر بیندازد و سازمان تا زمان پرداخت مالیات و عوارض فروش این نوع معاملات توسط کارفرما یا خریدار، مودی را مشمول جریمه تاخیر در پرداخت نخواهد کرد. در خصوص معاملات مذکور، تا زمان پرداخت مالیات و عوارض توسط خریدار، اعتبار مالیاتی برای وی از این بابت منظور نخواهد شد.

بنابراین، تاریخ تعلق مالیات در معاملات غیرنقدی همان تاریخ صدور صورتحساب است...assistant

ماده ۴ قانون مالیات بر ارزش افزوده می گوید: در معاملات غیرنقدی نظیر فروش اقساطی و اجاره به شرط تملیک و قراردادهای پیمانکاری و مشاوره ای، تاریخ تعلق مالیات و عوارض همان تاریخ صدور صورتحساب است؛ لکن مودی با رعایت تبصره فوق مجاز است پرداخت مالیات و عوارض فروش این نوع معامل

In [ ]:
# launch gradio app
iface.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://79f0b1134155bd76fc.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


[56 11  0]
